In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

In [4]:
import matplotlib as mlp

plt.style.use('dark_background')

mlp.rcParams['lines.linewidth'] = 2

mlp.rcParams['xtick.major.size'] = 15
mlp.rcParams['xtick.major.width'] = 2
mlp.rcParams['xtick.labelsize'] = 15
mlp.rcParams['xtick.color'] = '#4DFC2D'

mlp.rcParams['ytick.major.size'] = 15
mlp.rcParams['ytick.major.width'] = 2
mlp.rcParams['ytick.labelsize'] = 15
mlp.rcParams['ytick.color'] = '#4DFC2D'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#ED6919'
mlp.rcParams['axes.labelcolor'] = '#ED6919'

In [7]:
df = pd.read_csv('data.csv')

In [8]:
df.protokol.value_counts()

КМ1     1917
МВ      1334
КМ3     1054
КМ5     1031
КМ4      732
КМ2      510
КМ6      431
ТМ9      324
ТМ3      219
ТМ1      156
ТМ5      104
ТМ6       95
СКМ3      89
СКМ5      79
ТМ2       73
СКМ4      47
КМ7       41
СКМ6      36
ТМ4       36
СКМ2      26
КМ8       20
СКМ1      13
MB         1
Name: protokol, dtype: int64

In [40]:
import re
pattern = re.compile("KM[0-9]")

# Название колонки - англ, в лямбде - русский
df['KM'] = df.protokol.str[:2].apply(lambda x: x == 'КМ').astype(int)

df.head()

,Дней стельности при событии,Дни доения при событии,protokol,struck,result,stage,KM
0,291,213,МВ,0.00,0.50,0,0
1,291,215,ТМ6,0.50,0.75,1,0
2,291,221,КМ4,0.75,0.00,2,1
3,0,294,КМ5,0.25,0.00,0,1
4,0,302,КМ4,0.25,0.25,1,1


In [41]:
df.KM.sum() == 1917 + 1054 + 1031 + 732 + 510 + 431 + 41 + 20

True

In [42]:
df.KM.value_counts()

1    5736
0    2632
Name: KM, dtype: int64

#### Проведем стат проверку гипотез

$H_0:\quad$  На первом этапе лечения протоколы типа *КМ* дают такое же качество, как и остальные протоколы в среднем

$H_1:\quad$ *КМ* дает меньше нулей в `result`, чем остальные протоколы в среднем

$\alpha = 0.05$

In [51]:
df_km = df[df.stage == 0].copy().reset_index().drop('index', axis=1)

df_km.head()

,Дней стельности при событии,Дни доения при событии,protokol,struck,result,stage,KM
0,291,213,МВ,0.00,0.5,0,0
1,0,294,КМ5,0.25,0.0,0,1
2,0,103,КМ4,1.00,0.0,0,1
3,287,246,КМ1,0.25,0.0,0,1
4,154,275,КМ6,0.50,0.0,0,1


In [54]:
df_km['result_bin'] = df_km.result.apply(lambda x: x != 0).astype(int)

df_km.result_bin.value_counts()

0    1656
1    1111
Name: result_bin, dtype: int64

In [55]:
km = df_km[df_km.KM == 1]
others = df_km[df_km.KM != 1]

print(km.shape[0], others.shape[0])

1565 1202


In [61]:
km.result_bin.mean(), others.result_bin.mean()

(0.47667731629392973, 0.3036605657237937)

Если смотреть просто по выборкам, то для *КМ* на первом этапе значение `result` чаще принимает значение 1 (т.е. реже принимает значение 0)

In [67]:
from statsmodels.stats.weightstats import ztest

# Воспользуемся z тестом для сравнения долей

print('P-value:', ztest(km.result_bin, others.result_bin, alternative='larger')[1])

P-value: 4.661574559575273e-21


P-уровень значимости стремится к нулю, что гораздо меньше нашего фиксированного $\alpha$

=> Отвергаем нулевую гипотезу о равенстве средних (i.e. равенство распределений)

#### Также сравним соотношения `result` для всех протоколов со `stage` = `result_bin` = 0

In [72]:
df['result_bin'] = df.result.apply(lambda x: x != 0).astype(int)

df_updated = df[df.KM == 0].copy()

df_updated.head()

,Дней стельности при событии,Дни доения при событии,protokol,struck,result,stage,KM,result_bin
0,291,213,МВ,0.00,0.50,0,0,1
1,291,215,ТМ6,0.50,0.75,1,0,1
10,40,157,ТМ2,0.25,0.00,0,0,0
22,243,259,ТМ4,0.25,0.00,0,0,0
26,42,250,ТМ3,0.25,0.25,0,0,1


In [70]:
df.groupby('protokol')